In [1]:
import numpy as np
import pandas as pd
import math

In [2]:
def Estep(data, G, prob, mu, Sigma):
    g = np.zeros((len(data), G))
    U, D, _ = np.linalg.svd(Sigma)
    D_tilda = np.diag(1 / np.sqrt(D))
    x_tilda = D_tilda.dot(U.T.dot(data.T)).T # n by dim
    mu_tilda = D_tilda.dot(U.T.dot(mu)).T # G by dim
    for i in range(G):
        g[:, i] = np.sum(((x_tilda - mu_tilda[i])*(x_tilda - mu_tilda[i])), axis=1)
        g[:, i] = np.exp(-0.5 * g[:, i]) * prob[i]
    g /= np.sqrt((2*math.pi) ** ndim * np.linalg.det(Sigma))
    g /= np.sum(g, axis=1, keepdims=True)
    return g

In [3]:
def Mstep(data, g):
    prob = np.mean(g, axis=0)
    mu = np.zeros((ndim, G))
    Sigma = np.zeros((ndim, ndim))
    for i in range(G):
        mu[:, i] = np.sum(np.multiply(g[:, i:i+1], data), axis=0) / np.sum(g[:, i])
        temp_data = data - mu[:, i]
        temp_data1 = np.multiply(g[:, i:i+1], temp_data)
        Sigma += temp_data.T.dot(temp_data1)
    Sigma /= n
    return prob, mu, Sigma

In [4]:
def loglik(data, prob, mu, Sigma):
    g = np.zeros((len(data), G))
    U, D, _ = np.linalg.svd(Sigma)
    D_tilda = np.diag(1 / np.sqrt(D))
    x_tilda = D_tilda.dot(U.T.dot(data.T)).T # n by dim
    mu_tilda = D_tilda.dot(U.T.dot(mu)).T
    for i in range(G):
        g[:, i] = np.sum(((x_tilda - mu_tilda[i])*(x_tilda - mu_tilda[i])), axis=1)
        g[:, i] = np.exp(-0.5 * g[:, i]) * prob[i]
    g /= np.sqrt((2*math.pi) ** ndim * np.linalg.det(Sigma))
    llh = np.sum(np.log(np.sum(g, axis=1)), axis=0)
    return llh

In [5]:
def myEM(data, G, prob, mu, Sigma, itmax=20):
    for i in range(itmax):
        g = Estep(data, G, prob, mu, Sigma)
        prob, mu, Sigma = Mstep(data, g)
    return prob, mu, Sigma, loglik(data, prob, mu, Sigma)

In [6]:
data = pd.read_table("faithful.dat", sep="\s+", index_col=0)

In [7]:
n = len(data)
G = 2
ndim = 2
prob = np.array([10/n, 1-10/n])
mu = np.array([np.mean(data[:10], axis=0), np.mean(data[10:], axis=0)])
mu = mu.T
Sigma = np.zeros((2,2))
temp_data1 = np.array(data[:10]-mu[:, 0])
temp_data2 = np.array(data[10:]-mu[:, 1])
Sigma[0, 0] = np.sum(np.multiply(temp_data1[:,0], temp_data1[:,0])) + np.sum(np.multiply(temp_data2[:,0], temp_data2[:,0]))
Sigma[0, 1] = Sigma[1, 0] = np.sum(np.multiply(temp_data1[:,0],temp_data1[:,1]))+np.sum(np.multiply(temp_data2[:,0],temp_data2[:,1]))
Sigma[1, 1] = np.sum(np.multiply(temp_data1[:,1], temp_data1[:,1])) + np.sum(np.multiply(temp_data2[:,1], temp_data2[:,1]))
Sigma /= n

In [9]:
new_prob, new_mu, new_Sigma, llh = myEM(data, G, prob, mu, Sigma, itmax=20)

In [10]:
print(new_prob)
print(new_mu)
print(new_Sigma)
print(llh)

[0.04297883 0.95702117]
[[ 3.49564188  3.48743016]
 [76.79789154 70.63205853]]
           eruptions     waiting
eruptions   1.297936   13.924336
waiting    13.924336  182.580092
-1289.5693549424107


In [11]:
n = len(data)
G = 3
ndim = 2
prob = np.array([10/n, 20/n, 1-10/n-20/n])
mu = np.array([np.mean(data[:10], axis=0), np.mean(data[10:30], axis=0),
np.mean(data[30:], axis=0)])
mu = mu.T
Sigma = np.zeros((ndim,ndim))
temp_data1 = np.array(data[:10]-mu[:, 0])
temp_data2 = np.array(data[10:30]-mu[:, 1])
temp_data3 = np.array(data[30:]-mu[:, 2])
Sigma[0, 0] = np.sum(np.multiply(temp_data1[:,0], temp_data1[:,0])) + np.sum(np.multiply(temp_data2[:,0], temp_data2[:,0])) + np.sum(np.multiply(temp_data3[:,0], temp_data3[:,0]))
Sigma[0, 1] = Sigma[1, 0] = np.sum(np.multiply(temp_data1[:,0],temp_data1[:,1]))+np.sum(np.multiply(temp_data2[:,0],temp_data2[:,1]))+np.sum(np.multiply(temp_data3[:,0],temp_data3[:,1]))
Sigma[1, 1] = np.sum(np.multiply(temp_data1[:,1], temp_data1[:,1])) + np.sum(np.multiply(temp_data2[:,1], temp_data2[:,1])) + np.sum(np.multiply(temp_data3[:,1], temp_data3[:,1]))
Sigma /= n

In [13]:
new_prob, new_mu, new_Sigma, llh = myEM(data, G, prob, mu, Sigma, itmax=20)

In [14]:
print(new_prob)
print(new_mu)
print(new_Sigma)
print(llh)

[0.04363422 0.07718656 0.87917922]
[[ 3.51006918  2.81616674  3.54564083]
 [77.10563811 63.35752634 71.25084801]]
           eruptions     waiting
eruptions   1.260158   13.511538
waiting    13.511538  177.964191
-1289.350958862738
